# The Good Wife Generative Model (Cleaned)

**Topic:** The Good Wife Review Generation
<br>
**Goal:** To generate text resembling reviews of episodes of The Good Wife 
<br>


**Table of contents**


1. Overview
2. Importing the libraries
3. Pulling in the data
4. Dataset preperation
5. Modeling
6. Presenting examples


## 1. Overview

Here, I will parse a blog and try to generate reviews in the same style regarding the good wife.  This will be done at a character level to begin with.

## 2. Library Imports

In [1]:
#start pulling data
from bs4 import BeautifulSoup
import requests
from lxml import html
import os
import pickle
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import re

## 3. Pulling Data

Let's start with the rewatch links.

In [2]:
sitedoc = requests.get('https://florrickandassociates.tumblr.com/RewatchProject')
#sitedoc2 = html.fromstring(sitedoc.content)

soup = BeautifulSoup(sitedoc.text, 'html.parser')
#print(soup.prettify())

#print(soup.get_text())

#example of gettinga all links
for link in soup.find_all('a'):
    print(link.get('href'))

/
None
/allshows
/meta
/twitter
/tgfguide
/ask
http://florrickandassociates.tumblr.com/post/89661393894/tgw-rewatch-project-1-1x01-pilot
http://florrickandassociates.tumblr.com/post/89673346174/tgw-rewatch-project-2-1x02-stripped
http://florrickandassociates.tumblr.com/post/89695900852/tgw-rewatch-project-3-1x03-home
http://florrickandassociates.tumblr.com/post/89773078489/tgw-rewatch-project-4-1x04-fixed
http://florrickandassociates.tumblr.com/post/89794592729/tgw-rewatch-project-5-1x05-crash
http://florrickandassociates.tumblr.com/post/89998482039/tgw-rewatch-project-6-1x06-conjugal
http://florrickandassociates.tumblr.com/post/90080429879/tgw-rewatch-project-7-1x07-unorthodox
http://florrickandassociates.tumblr.com/post/90406089489/tgw-rewatch-project-8-1x08-unprepared
http://florrickandassociates.tumblr.com/post/90481876889/tgw-rewatch-project-9-1x09-threesome
http://florrickandassociates.tumblr.com/post/90485995864/tgw-rewatch-project-10-1x10-lifeguard
http://florrickandassociates.

Now that all of the links have been seen, we can pull them into a list.  

In [23]:
#make this a list 
lnklst = []
for link in soup.find_all('a'):
    lnklst.append(link.get('href'))
lnklst = [x for x in lnklst[5:] if 'tgw-rew' in x]

Now, let's pull the links for other reviews.

In [29]:
sitedoc = requests.get('https://florrickandassociates.tumblr.com/meta')
soup = BeautifulSoup(sitedoc.text, 'html.parser')


#make this a list 
lnklst2 = []
for link in soup.find_all('a'):
    lnklst2.append(link.get('href'))
lnklst2 = [x for x in lnklst2[5:] if 'tgw-thought' in x]

Simply add the two lists - these are the links to all of the relevant reviews. Below, we remove one errant list from another website for the purposes of simplicity.

In [30]:
lnklst = lnklst+lnklst2

In [34]:
lnklst.remove('https://t.umblr.com/redirect?z=https%3A%2F%2Fflorrickandassociates.wordpress.com%2F2015%2F10%2F30%2Ftgw-thoughts-7x04-taxed%2F&t=NzcxMzY3MzM0ZGRkYzIzNmNkZmZiNDlmNzY3NTBkNmM4MThiMjlkMyxUbU4zdWpESg%3D%3D&p=&m=0')

Now, let's parse each page...

Commment out the soup.prettify() when rerunning, as it creates quite a long output!

In [ ]:
sitedoc = requests.get(lnklst[40])
soup = BeautifulSoup(sitedoc.text, 'html.parser')

#print(soup.prettify())

Below, we can parse this to get the actual context - as an example.

In [71]:
rr = soup.find('div', class_="post-content").find_all('li')#.get_text()
rr = [a.text for a in rr]
' '.join(rr)

'We pick up where we left off, which means we have to experience the heartbreak all over again.\xa0 I really wanted that. Kalinda tries to call Alicia to tell her first, but she can’t make the words come out of her mouth. Alicia can tell that something’s wrong, but poor Kalinda, who loves Alicia so much and would never want to hurt her, doesn’t know how to say what’s on her mind. It would be a difficult situation for anyone to be in; it’s a thousand times more difficult for someone with Kalinda’s personality. I don’t know why, but I remember this case a lot more clearly than most cases on this show. I think it’s the song (Thicky Trick was not TGW’s first earworm!). Marissa’s back! Love her! “America does not suck. People suck,” proclaims Eli. I love the way Eli and Marissa talk. Eli tells Diane she should consider (again) running for a judgeship. During s4/s5, I remembered very clearly that Diane had been approached to run for a judgeship, but I didn’t remember that Eli also brought th

Let's try one that isn't rewatch.

In [72]:
sitedoc = requests.get(lnklst[145])
soup = BeautifulSoup(sitedoc.text, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<script>
 var __pbpa = true;
</script>
<script>
 var translated_warning_string = 'Warning: Never enter your Tumblr password unless \u201chttps://www.tumblr.com/login\u201d\x0ais the address in your web browser.\x0a\x0aYou should also see a green \u201cTumblr, Inc.\u201d identification in the address bar.\x0a\x0aSpammers and other bad guys use fake forms to steal passwords.\x0a\x0aTumblr will never ask you to log in from a user\u2019s blog.\x0a\x0aAre you absolutely sure you want to continue?';
</script>
<script language="javascript" src="https://assets.tumblr.com/assets/scripts/pre_tumblelog.js?_v=b9f848c06fcba7eaf305d4a7cb7a1b98" type="text/javascript">
</script>
<!DOCTYPE html>
<html class="no-js">
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# blog: http://ogp.me/ns/blog#">
  <!-- DESIGNED BY LUMIBOX.CO.UK-->
  <!-- Update 14.11.14: fixed line-height and font-weight issues -->
  <meta content="" name="text:Disqus Shortname"/>
  <title>
   makin' my own

In [73]:
rr = soup.find('div', class_="post-content").find_all('li')#.get_text()
rr = [a.text for a in rr]
' '.join(rr)

'The elevator doors open on the LAL\nreception desk. “Welcome to Lockhart, Agos, and Lee,” Diane narrates. “As you\nprobably know, we are an all-service law firm with a rich tradition. The firm\nwas formed twelve years ago by Jonas Stern, and myself, Diane Lockhart.” Only\ntwelve years ago? Formed by Stern and Diane, but not Will? Continuity errors\nabound. Or maybe not—Stern/Diane founding and Will joining later was the\nnarrative up until season five. Whatever. Diane is looking for a narrative\nhere, not accuracy. (Also, LOL at that portrait of Stern.) “Last year, the firm reconstituted\nitself with Cary Agos and David Lee as name partners,” Diane says, omitting the\npart where… well, the entire series happens. I presume you’ve watched it, so I\nwon’t recap it. “We occupy two floors in the\nDryden building: the executive suite on the 28th floor and the 27th\nfloor, where support staff and most associates work,” Diane explains. Hey!\nStock footage! It’s like the good old days! Also, t

Looks like there's newline characters but otherwise we can deal with it.

So, below, we will loop through each of the links, pull in the values into a beautiful soup, and parse the text into a list.

In [85]:
textlist = []
for link in lnklst:
    linkuse = requests.get(link)
    linksoup = BeautifulSoup(linkuse.text, 'html.parser')
    rr = linksoup.find('div', class_="post-content").find_all('li')#.get_text()
    rr = [a.text for a in rr]
    [textlist.append(item)  for item in rr]    
    print(link)
    print(rr[0])


http://florrickandassociates.tumblr.com/post/89661393894/tgw-rewatch-project-1-1x01-pilot
I love the opening scene. Wonderfully shot, wonderfully edited, wonderfully acted. The slow-motion, combined with the different images and the music, creates a sense of disorientation, mimicking Alicia’s feelings as she steps reluctantly into the spotlight. This scene also does a terrific job of demonstrating that it’s Alicia we’re meant to focus on—we see her POV, her in relation to the audience, and the little details she notices like the piece of lint on Peter’s sleeve. This is all the more impressive because…
http://florrickandassociates.tumblr.com/post/89673346174/tgw-rewatch-project-2-1x02-stripped
Cary actually sent Alicia the link to the video of the news report about Peter. Seriously, dude?
http://florrickandassociates.tumblr.com/post/89695900852/tgw-rewatch-project-3-1x03-home
I love the flashbacks in this episode. They’re like the trial version of memory pops—subjective and a bit trivia

http://florrickandassociates.tumblr.com/post/92492950964/tgw-rewatch-project-22-1x22
This episode picks up right where the last one ended, but, because it’s a long sequence, we get to see it from different perspectives—we see Kalinda and Julius talking instead of Cary being fired.
http://florrickandassociates.tumblr.com/post/92541352894/tgw-rewatch-project-23-1x23-running
The Florrick family goes out to dinner! I wish we got more of these scenes. They’re really nice, and, at least when things are going well, everyone seems to be having a good time. They’re talking, catching up on things, telling stories… the show should remind us more often of that.
http://florrickandassociates.tumblr.com/post/92654879709/tgw-rewatch-project-24-2x01-taking-control
Because it’s the start of a new season, I’m going to put a disclaimer here: These Rewatch Project entries contain spoilers.
http://florrickandassociates.tumblr.com/post/92739640869/tgw-rewatch-project-25-2x02-double-jeopardy
Matan is ALSO in 

http://florrickandassociates.tumblr.com/post/96360233904/tgw-rewatch-project-47-3x01-a-new-day
One of my biggest criticisms of the criticism that TGW gets is the argument that the title should be different because Alicia’s not literally “Good”. People love to point this out like they’ve noticed a huge discord between the show’s intentions and what makes it to the screen, when really, what they’ve noticed is the point of the show. They somehow haven’t figured out that the title is meant to be ironic and call out unrealistic societal expectations.  Anyway, I’m complaining about this now because the show does itself no favors on this subject by opening season three with “Baby Did a Bad, Bad Thing.” One of the rare song choices I don’t like on this show.
http://florrickandassociates.tumblr.com/post/96391219669/tgw-rewatch-project-48-3x02-the-death-zone
“Do you ever get the feeling we’re losing control here?” “All the time.” Diane and Will worry about the distribution of power at L/G.
http:

http://florrickandassociates.tumblr.com/post/97255167014/tgw-rewatch-project-71-4x03-two-girls-one
The episode kicks off at Synth Magazine, where a reporter named Mandy Post is putting together a story on Peter and Alicia. “Come on Petunia” by The Blow plays over the scene, only it has different lyrics than the version you can buy on iTunes or find anywhere online. I wish I could get it with the TGW lyrics, which begin, “Can everybody see that I’m on fire?”
http://florrickandassociates.tumblr.com/post/97272067329/tgw-rewatch-project-72-4x04-dont-haze-me-bro
Jackie leaves the hospital at the beginning of the episode. I find it incredibly amusing she has a pin of a bee or a wasp that she wears rather frequently.
http://florrickandassociates.tumblr.com/post/97324577409/tgw-rewatch-project-73-4x05-waiting-for-the
It’s a Bishop episode, so we need a previously on. Yay.
http://florrickandassociates.tumblr.com/post/97328253189/tgw-rewatch-project-74-4x06-the-art-of-war
The episode begins with

http://florrickandassociates.tumblr.com/post/97932872704/tgw-rewatch-project-101-5x11-goliath-and-david
C’mon, sing along with me: That lil mama thicky thick, but she ain’t nothin’ but a thicky trick. That lil mama thicky thick, but she ain’t nothin’ but a thicky trick. Thicky thick, tricky trick, she a thicky thick tricky trick.
http://florrickandassociates.tumblr.com/post/97935517164/tgw-rewatch-project-102-5x12-we-the-juries
I don’t like this episode at all. But at least we get Matan and Geneva!
http://florrickandassociates.tumblr.com/post/97980776819/tgw-rewatch-project-103-5x13-parallel
This episode starts out with a previously… for AMC’s Darkness at Noon. Can you tell that the Kings hate previouslies even more than I do? And that they don’t think that clichés, lines that sound profound (“THERE ARE LINES AND THEN THERE ARE LINES”), and excessive violence are the hallmarks of good television? I love Darkness at Noon.
http://florrickandassociates.tumblr.com/post/97992691079/tgw-rewa

http://florrickandassociates.tumblr.com/post/107783115039/tgw-thoughts-6x11-hail-mary
And we’re back! Wasn’t that a fun hiatus?!
http://florrickandassociates.tumblr.com/post/108269500559/tgw-thoughts-6x12-the-debate
The Debate starts with a statement about its contents, so I’m going to start my review with one, too. Since I’ll undoubtedly be writing about race here, I think it’s an appropriate time to give all of you a few biographical details about myself so you can put my remarks in context if you so desire. I’ll say upfront that I don’t think I’m qualified in any way to write about racial issues and that I would much rather read other people’s thoughts on these topics than share my own (that is to say: please link me to any eloquent write-ups of the show’s diversity issues that you find). The thoughts you’re reading here belong to a straight, white, moderately well-informed, upper-middle class female college student, and you can feel free to make of that what you will.
http://florri

http://florrickandassociates.tumblr.com/post/131826657979/tgw-thoughts-7x03-cooked
The episode kicks off with a
close-up of Howard Lyman, and my level of enthusiasm instantly drops. But not
for long! Lyman, it turns out, is sitting in Alicia’s apartment, and now I’m
excited again. Alicia walks in the door and begins to walk towards her office
when Lyman catches her off guard.
http://florrickandassociates.tumblr.com/post/133226915459/tgw-thoughts-7x06-lies
The episode opens with a woman
taking a polygraph test with questions reminiscent of the ones we saw in 5x18 (All Tapped Out, the one where the series
regulars discover the NSA’s listening in). Turns out there’s a reason for that.
http://florrickandassociates.tumblr.com/post/133755542459/tgw-thoughts-7x07-driven
The episode opens with an out of
focus shot of what we later realize is the LAL conference room. A red toy car
crosses the screen, in focus. This is a deposition; Canning is representing the
woman suing. As we find out, the is

http://florrickandassociates.tumblr.com/post/144026426569/tgw-thoughts-7x21-verdict
Welcome to the penultimate episode
of the series, everyone!
http://florrickandassociates.tumblr.com/post/144493151409/tgw-thoughts-7x22-end
The episode starts in chaos.
Alicia, Eli, Peter, and Lucca are in a car, each of them absorbed in his or her
own phone conversation. (Lucca’s working on the case, Peter and Eli are talking
politics, and Alicia’s speaking with Grace.) Hey guys! Wear your seatbelts!


This was surprisingly quick. 

## 4. Dataset preparation

How many jpegs and gifs are in there?

In [87]:
[x for x in textlist if '.gif' in x]

['Alicia laughs and sits down in her\nchair… which promptly breaks. Hey, where have I seen that before? http://25.media.tumblr.com/tumblr_lyydxkwbrq1qzkueso1_500.gif']

In [88]:
[x for x in textlist if '.jpg' in x]

['When they call the vote again,\nAlicia votes so hesitantly it’s obvious something’s up. Look at that\ndetermination: http://juliannamargulies.org/photos/albums/Television/The%20Good%20Wife/S7/caps/707/TGW707Caps_041.jpg',
 'Alicia’s kinda unsettling when she\nlooks directly into the camera, isn’t she? http://juliannamargulies.org/photos/albums/Television/The%20Good%20Wife/S7/caps/707/TGW707Caps_112.jpg',
 'One of the extras is wearing a\nsweater I’m 99% sure used to be part of Robyn’s wardrobe. Scratch that, I’m\n100% sure: http://www.spoilersguide.com/media/cache/300x189/wp-content/uploads/2014/05/The-Good-Wife-5x22-1.jpg.\nBRING BACK ROBYN! BRING BACK ROBYN! \xa0',
 'Back from break, Diane is\nlecturing Lucca about how she failed to follow orders. Lucca sits on a couch.\nDiane and Cary tower over her. Alicia stands in the background. See? http://images.springfieldspringfield.co.uk/screencaps/the-good-wife-2009/season7/s07e14/s07e14_438.jpg\nPower dynamics.',
 '(You’ll recall this, 

Not too many.

Next, I'm going to pickle textlist so I can restart the kernel and not worry about scraping again.

The first commented out line is the dump - the next is reloading it!

In [3]:
#pickle.dump( textlist, open( "textlist.p", "wb" ) )
textlist = pickle.load( open( "textlist.p", "rb" ) )

Now, we can continue to clean by making this one long document, and replacing some special characters.

In [4]:
text = ' '.join(textlist).replace('\n', ' ').replace('\xa0', ' ')

Let's see how long it is.

In [6]:
#word count
len(text.split(' '))

477520

Now, let's remove all links.

In [7]:
re.findall(r'http\S+', text)

['http://juliannamargulies.org/photos/albums/Television/The%20Good%20Wife/S7/caps/707/TGW707Caps_041.jpg',
 'http://juliannamargulies.org/photos/albums/Television/The%20Good%20Wife/S7/caps/707/TGW707Caps_112.jpg',
 'http://www.spoilersguide.com/media/cache/300x189/wp-content/uploads/2014/05/The-Good-Wife-5x22-1.jpg.',
 'http://www.gettyimages.com/music/artists/91963-dj-troye).',
 'http://25.media.tumblr.com/tumblr_lyydxkwbrq1qzkueso1_500.gif',
 'http://images.springfieldspringfield.co.uk/screencaps/the-good-wife-2009/season7/s07e14/s07e14_438.jpg',
 'http://juliannamargulies.org/photos/albums/Television/The%20Good%20Wife/S4/caps/418/4x18_10000919.jpg.',
 'http://www.ew.com/article/2012/04/29/good-wife-season-finale-kalinda-husband),']

In [8]:
text = re.sub(r'http\S+', '', text)

Now to start with a simple, character based example for generating text, based in part on some great online tutorials!  First, make it all lowercase, and review the characters in the file.

In [12]:
text = text.lower()

In [13]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


75 unique characters


We can see that there's a lot of characters, let me see what they are...

In [14]:
vocab

[' ',
 '!',
 '#',
 '$',
 '%',
 '&',
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 ']',
 '^',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '~',
 '©',
 'ç',
 'é',
 'ï',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '…',
 '′']

Most of these make sense, so I'll proceed to vectorize.

In [16]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
#the above makes a dictionary for the vocab, and below we make a vocab aray
idx2char = np.array(vocab)
#text as int allows us to have an intiger vector of the characters
text_as_int = np.array([char2idx[c] for c in text])


Now, let's start processing this, and thinking about our batches.  These batches will be parts of the text divided up, to properly learn from.

In [20]:
# The maximum length sentence we want for a single input in characters
seq_length = 100 #decided not to change this, to allow more batches for learning
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Now, let's look at the first batch for a good example...

In [21]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'i love the opening scene. wonderfully shot, wonderfully edited, wonderfully acted. the slow-motion, c'
'ombined with the different images and the music, creates a sense of disorientation, mimicking alicia’'
's feelings as she steps reluctantly into the spotlight. this scene also does a terrific job of demons'
'trating that it’s alicia we’re meant to focus on—we see her pov, her in relation to the audience, and'
' the little details she notices like the piece of lint on peter’s sleeve. this is all the more impres'


Now, we are basically going to make a large number of datapoints by shifting, and using 100 characters to predict the 101st for all but 100 characters.

In [22]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Let's look at an example.

In [23]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))


Input data:  'i love the opening scene. wonderfully shot, wonderfully edited, wonderfully acted. the slow-motion, '
Target data: ' love the opening scene. wonderfully shot, wonderfully edited, wonderfully acted. the slow-motion, c'


Now we can create training batches.

In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset


<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

## 5. Modelling

Now, building the model itself.  While I'm used to embeddings meaning something different, here it seems to be the size of the input layer - and corresponds to a trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions; GRU is the key RNN component.  Here, we have 1024 units.  We specify that it should return the full sequence and not just the last item in it, and use the last output of the node before it as the first input of the next node.  There's also an initializer for the weights. Then, we have a dense layer as an output.

Possible changes to make later - what if we did an lstm?  Which parameters of the GRU can and should be changed?


In [25]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension -
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

#this is code for building a model
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    #the sequential shows that we are putting them in order.  Embedding takes our vocab as a dimension, and then the number
    #of vectors for a count, and our batch size.
    #the GRU is the RNN option, we could also use an LSTM if we wanted to.  it is returning sequences because 
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

Now, let's build the model.  Note that this is building the network, it hasn't been compiled or trained yet.

In [26]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

There is a good example in the source about how this network looks.  Now, we can run the model.  Let's check the output shape first.

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 75) # (batch_size, sequence_length, vocab_size)


Now, let's look at the model shape.

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           19200     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 75)            76875     
Total params: 4,034,379
Trainable params: 4,034,379
Non-trainable params: 0
_________________________________________________________________


To get predictions from the model we'll need to sample from the output distribution.  This will allow us to get the indices of the character. This distribution is defined by the logits over the character vocabulary.

In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
#decode the indices
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'othing. will ends up arguing in front of some international sports law panel that’s in a stadium and'

Next Char Predictions: 
 '$‘:’!c4”6–vékï%n^]>‘o/1|–”t&q…u_@7.1ghdïef#gn[o+(ai@;:[1–q—d45k”.>_”l#ï9;a%8“=–8*”t/$_g0!/$,x=^=8&1)'


Of course these make no sense since this has not been trained yet.

Now we add a loss function (categorical crossentropy is the standard) and an optimizer for how we adjust learning rate.

In [30]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 75)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.319801


Below we compile, and prep checkpoints to save.

In [32]:
#use adam to compile
model.compile(optimizer='adam', loss=loss)

#add checkpoints for model
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Let's try training with 10 .  This will be slow, as I'm training it locally, with no GPU or anything.

In [33]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
417/417 [==============================] - 1255s 3s/step - loss: 2.0854
Epoch 2/10
417/417 [==============================] - 1205s 3s/step - loss: 1.4123
Epoch 3/10
417/417 [==============================] - 1347s 3s/step - loss: 1.2745
Epoch 4/10
417/417 [==============================] - 1365s 3s/step - loss: 1.2109
Epoch 5/10
417/417 [==============================] - 1368s 3s/step - loss: 1.1697
Epoch 6/10
417/417 [==============================] - 1385s 3s/step - loss: 1.1378
Epoch 7/10
417/417 [==============================] - 1398s 3s/step - loss: 1.1109
Epoch 8/10
417/417 [==============================] - 1346s 3s/step - loss: 1.0868
Epoch 9/10
417/417 [==============================] - 1476s 4s/step - loss: 1.0647
Epoch 10/10
417/417 [==============================] - 1285s 3s/step - loss: 1.0443


That took a long time!  Now, we can take the final checkpoint, and reload the next day, as below:

In [34]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [35]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            19200     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 75)             76875     
Total params: 4,034,379
Trainable params: 4,034,379
Non-trainable params: 0
_________________________________________________________________


THis evaluates and generates.  We pick a starting string and generate a set sequence.  Using the conversion to vecor from above, it samples at each step from the generated distribution.  It treats batch size as 1 rather than 64.

## 6. Prediction

In [112]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.2

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate): #for each of the 1000 we want to output
        predictions = model(input_eval) #we will build the odel with the input
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature #scale predictions it seems
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy() #this looks like sampling

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0) #then we seem to add the predicted on into our input eval

        text_generated.append(idx2char[predicted_id]) #and append the text generated

    return (start_string + ''.join(text_generated)) #finally we output the text generated

Let's try a sample.

In [85]:
print(generate_text(model, start_string=u"alicia and peter"))

alicia and peter are an associate that was a little bit. the writers are going to be a lawyer and the writers wanted to be with him. the writers are good at the same time they can get a sense of the show in the past and the writers are going to be a part of the partners and alicia and peter are an alicia that the writers are already over every single thing to do with the fantasy that the writers are going to be a part of the partners and alicia and peter are an alicia that they want to be with her hands and the


This works!  Doesn't make too much sense but these are generally sentences.

In [88]:
print(generate_text(model, start_string=u"my favorite thing about the good wife is"))

my favorite thing about the good wife is that the writers are good at the same time that i don’t know if alicia was just a little bit of the office and then says, “i don’t know if i want to be waiting for a moment, and then they can get a sense of the show that the writers are going to be a lawyer and the writers would be a consistent thing to do with that one where the writers wanted to be the one who was in the middle of the frame. she was already on the other hand, i don’t know if i can think of is that it’s a bit of a relationship


In [100]:
print(generate_text(model, start_string=u"the writers"))

the writers are going to be a lawyer and he was already on the other hand, i don’t know if alicia wants to know if alicia was supposed to be a part of the partners and alicia and peter are an associate that was a little bit of the office. she walks away. “you were a little bit, but i don’t know if i can see you and i don’t know if i can get a bit of a relationship with him.” yes, peter and alicia are all sure what the writers wanted to be a lawyer. i don’t know if they were a little bit of the office. she 


In [106]:
print(generate_text(model, start_string=u"case stuff happens"))

case stuff happens. i like that they were seeing alicia and peter are an associate who was supposed to be a part of the office and then alicia and peter are an associate that was a little bit of the office and then reading a bit of a relationship with jason. “i don’t know if i can see you this way of a client and the writers want to be the one who was in the middle of the florrick apartment, which is a way of doing that the writers are going to be a part of the office to talk to him about the very least favorite 


In [117]:
print(generate_text(model, start_string=u"fucking"))

fucking to talk to alicia. the last thing i have to be a bad mood to think of that eli says he was just a little bit, but it could have been a little bit of the office. alicia and peter are an associate who was supposed to be a lawyer and i don’t know if they have a little bit of a constant problem with that one and being a walk of a relationship with his time to say that the writers are still playing out a lot of the office. the client of the series regulars that was a little bit. “i don’t know if ali
